# Computational Social Science Project #2 

*Group number:* 

*Group members:*   
Rachel Pizatella-Haswell, Brenda Sciepura, Omair Gil

*Semester:* Fall 2023


Below we fill in some of the code you might use to answer some of the questions. Here are some additional resources for when you get stuck:
* Code and documentation provided in the course notebooks  
* [Markdown cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) to help with formatting the Jupyter notebook
* Try Googling any errors you get and consult Stack Overflow, etc. Someone has probably had your question before!
* Send me a pull request on GitHub flagging the syntax that's tripping you up 

## 1. Introduction/Setup

#### a) Import relevant libraries
Add the other libraries you need for your code below and/or as you go. 

In [ ]:
# import libraries you might need here 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import geopandas as gpd

#Import specific sklearn functions
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV

# use random seed for consistent results 
np.random.seed(273)

#### b) Read in and inspect data frame 
Read in the data frame and look at some of its attributes. 

In [ ]:
os.getcwd()

In [ ]:
diabetes = pd.read_csv("Diabetes with Population Info by County 2017.csv", 
                       #CountyFips needs to be a string so leading 0 isn't dropped (this is only if you want to make choropleth map): 
                       dtype={"CountyFIPS": str}) 

In [ ]:
diabetes

In [ ]:
# look at the dimensions of the diabetes data frame
print('shape: ', diabetes.shape) 

In [ ]:
pd.set_option('display.max_rows', 100) # tells pandas how many rows to display when printing so results don't get truncated

# look at the data types for each column in diabetes df 
print('data types:', diabetes.dtypes)

Immediately, we see that some of the features that should be numeric (e.g., Diabetes_Number, Obesity_Number,  and Physical_Inactivity_Number) are not. We can check to see what the non-numeric values are in a column where we are expecting numeric information with a combination of `str.isnumeric()` and `unique()`.

In [ ]:
# Return rows where the column "Diabetes_Number" is non-numeric and get the unique values of these rows
# the "~" below in front of diabetes negates the str.isnumeric() so it only takes non-numeric values
print(diabetes[~diabetes["Diabetes_Number"].str.isnumeric()]["Diabetes_Number"].unique()) 

In [ ]:
diabetes[~diabetes["Diabetes_Number"].str.isnumeric()]

In [ ]:
# Now do the same as above, but for "Obesity_Number" :
print(diabetes[~diabetes["Obesity_Number"].str.isnumeric()]["Obesity_Number"].unique()) 

In [ ]:
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)']

The values contained in the two columns above making them objects (rather than integers) appear to be strings like "No Data" and "Suppressed." Let's drop those rows in the next section, and also recode Physical_Inactivity_Number to be an integer. 

#### c. Recode variables

Convert 'Diabetes_Number', 'Obesity_Number', and 'Physical_Inactivity_Number' to integers below so we can use them in our analysis. Also fill in the object type we want to recode 'sex and age_total population_65 years and over_sex ratio (males per 100 females)' to. 

In [ ]:
# Diabetes
# keep only useful info about our target feature, i.e., where diabetes_number not = 'Suppressed'
diabetes = diabetes[diabetes['Diabetes_Number']!="Suppressed"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 

In [ ]:
assert not (diabetes['Diabetes_Number'] == 'Suppressed').any()

In [ ]:
diabetes = diabetes[diabetes['Obesity_Number']!='No Data']

In [ ]:
diabetes[['Diabetes_Number', 'Obesity_Number', 'Physical_Inactivity_Number']] = diabetes[['Diabetes_Number', 'Obesity_Number', 'Physical_Inactivity_Number']].astype(int) 

In [ ]:
diabetes[['Diabetes_Number', 'Obesity_Number', 'Physical_Inactivity_Number']].dtypes

In [ ]:
# 65+ sex ratio had one "-" in it so let's drop that row first
diabetes = diabetes[diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)']!= "-"]

In [ ]:
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'].head()

In [ ]:
# change to numeric (specifically, integer or float?) from string (because originally included the "-" )
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'] = diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'].astype(float)

In [ ]:
diabetes['total housing units']

We should probably scale our count variables to be proportional to county population. We create the list 'rc_cols' to select all the features we want to rescale, and then use the `.div()` method to avoid typing out every single column we want to recode. 

In [ ]:
# select count variables to rc to percentages; make sure we leave out ratios and our population variable b/c these don't make sense to scale by population
rc_cols = [col for col in diabetes.columns if col not in ['County', 'State', 'CountyFIPS', 
                                                        'sex and age_total population_65 years and over_sex ratio (males per 100 females)', 'sex and age_total population_sex ratio (males per 100 females)', 'sex and age_total population_18 years and over_sex ratio (males per 100 females)',  
                                                        'race_total population', 'total housing units',]]

In [ ]:
diabetes[rc_cols] = diabetes[rc_cols].apply(pd.to_numeric, errors='coerce') # recode all selected columns to numeric

In [ ]:
# divide all columns but those listed above by total population to calculate rates
diabetes[rc_cols] = diabetes[rc_cols].div(diabetes['race_total population'], axis=0)

In [ ]:
diabetes

Let's check our work. Are all rates bounded by 0 and 1 as expected? 

In [ ]:
pd.set_option('display.max_columns', None)
# inspect recoded values
diabetes_summary = diabetes.describe().transpose() # note we use the transpose method rather than .T because this object is not a numpy array
diabetes_summary

In [ ]:
# check recoding 
with pd.option_context('display.max_rows', 100, 'display.max_columns', None): 
    display(diabetes_summary.iloc[ : ,[0,1,3,7]]) # select which columns in the summary table we want to present

In [ ]:
for col in rc_cols:
    condition = (diabetes[col] >= 0) & (diabetes[col] <= 1)
    assert condition.all(), f"Not all values in {col} are bounded by 0 and 1."

#### d. Check for duplicate columns

There are a lot of columns in this data frame. Let's see if there are any are duplicates. 

In [ ]:
# I used Google to figure this out, and adapted this example for our purposes:  
# source: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/ 
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicateColumnNames = list(getDuplicateColumns(diabetes))
print('Duplicate Columns are as follows: ')
duplicateColumnNames

In [ ]:
diabetes.loc[:,['hispanic or latino and race_total population',
 'sex and age_total population_18 years and over_1',
 'race_total population_two or more races_1',
 'sex and age_total population_65 years and over_1',
 'sex and age_total population',
 'race_total population_one race_1']]

In [ ]:
# now drop list of duplicate features from our df using the .drop() method
diabetes = diabetes.drop(columns=duplicateColumnNames) 

In [ ]:
# Define a dictionary mapping states to regions
state_to_region = {
    'Alabama': 'Southeast',
    'Alaska': 'West',
    'Arizona': 'West',
    'Arkansas': 'South',
    'California': 'West',
    'Colorado': 'West',
    'Connecticut': 'Northeast',
    'Delaware': 'Northeast',
    'District of Columbia': 'Southeast',
    'Florida': 'Southeast',
    'Georgia': 'Southeast',
    'Hawaii': 'West',
    'Idaho': 'West',
    'Illinois': 'Midwest',
    'Indiana': 'Midwest',
    'Iowa': 'Midwest',
    'Kansas': 'Midwest',
    'Kentucky': 'South',
    'Louisiana': 'South',
    'Maine': 'Northeast',
    'Maryland': 'Northeast',
    'Massachusetts': 'Northeast',
    'Michigan': 'Midwest',
    'Minnesota': 'Midwest',
    'Mississippi': 'South',
    'Missouri': 'Midwest',
    'Montana': 'West',
    'Nebraska': 'Midwest',
    'Nevada': 'West',
    'New Hampshire': 'Northeast',
    'New Jersey': 'Northeast',
    'New Mexico': 'West',
    'New York': 'Northeast',
    'North Carolina': 'Southeast',
    'North Dakota': 'Midwest',
      'Ohio': 'Midwest',
    'Oklahoma': 'South',
    'Oregon': 'West',
    'Pennsylvania': 'Northeast',
    'Rhode Island': 'Northeast',
    'South Carolina': 'Southeast',
    'South Dakota': 'Midwest',
    'Tennessee': 'South',
    'Texas': 'South',
    'Utah': 'West',
    'Vermont': 'Northeast',
    'Virginia': 'Southeast',
    'Washington': 'West',
    'West Virginia': 'South',
    'Wisconsin': 'Midwest',
    'Wyoming': 'West'
}

# Add a new 'Region' column based on the mapping
diabetes['Region'] = diabetes['State'].map(state_to_region)

In [ ]:
# Print to verify'Region' column has been added
diabetes

## 2. Exploratory Data Analysis

### Exploring geographic variation in diabetes prevalence
Because we are interested in understanding which counties to target for a diabetes prevention program, the geographic variation in diabetes prevalence is of particular importance. Depending on the aims of the policymaker, we may be interested in targeting regions that have multiple counties with high diabetes prevalence. Introducing the prevention program in a high risk region could have the benefit of better utilizing the fixed costs that would be necessary toget the program up and running in order eventually scale up and reach more counties. Furthermore, we might expect that the benefits of a successful program pilot could spill over to neighboring counties with similarly high diabetes prevalence. 

### Read in US county shapefile from the UC census and merge to the diabetes dataset

In [ ]:
gdf = gpd.read_file('cb_2018_us_county_500k.shp')

In [ ]:
gdf[gdf['STATEFP'] == "01"]

In [ ]:
gdf['CountyFIPS'] = gdf['STATEFP'] + gdf['COUNTYFP']

In [ ]:
merge_df = diabetes.merge(gdf)

In [ ]:
merged_df = gdf.set_index('CountyFIPS').join(diabetes.set_index('CountyFIPS'))

### Plot geographic variation in diabetes

In [ ]:
#Plot figure of diabetes percentage across counties
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
merged_df.plot(column='Diabetes_Number', cmap='magma_r', linewidth=0.01, ax=ax, edgecolor='0.8')
ax.axis('off')
plt.xlim(-130, -65)
plt.ylim(25, 55) 
ax.set_title('Diabetes Rate by U.S. Counties')
sm = plt.cm.ScalarMappable(cmap='magma_r', norm=plt.Normalize(vmin=merged_df['Diabetes_Number'].min(), vmax=merged_df['Diabetes_Number'].max()))
sm._A = []
cbar = plt.colorbar(sm)

# Set the label for the colorbar
cbar.set_label('Percentage Label')
fig.tight_layout()
plt.show()

### Get summary statistics for diabetes rates across counties

In [ ]:
diabetes['Diabetes_Number'].describe()

### Discussion
Diabetes varies significantly from county to county. County prevalence ranges from just 2% to 22%. The map shows that counties with high diabetes rates tend to be highly clustered. Such clusters exist throughout the midwest and western states, but counties with high diabetes rates are most prevalent in the south. Consequently, policymakers might want to focus on the southern states  for the roll out of the pilot program in order to learn how to combat diabetes in areas where it is most concentrated.

## Investigate relationship between Hispanic population and diabetes
The following chart shows how diabetes prevalence varies with the percentage of a county that is Hispanic or Latino. Understanding the demographic makeup of counties with high diabetes rate is crucial for treating and preventing the disease. Understanding if diabetes tends to be high among certain ethnic groups or ages can help policymakers to construct prevention programs that are culturally appropriate and that work in partnership with the right community organizations and leaders. 

The graph below shows a weak relationship between the percentage of a county that identifies as Hispanic/Latino and the diabetes rate. Furthermore, the correlation is slightly negative. This indicates that prevention of diabetes among Hispanic/Latino populations might not warrant special consideration for ploiycmakers. Note that this is an average of the relationship and that there are some counties with both large Hispanic/Latino poulations and high diabetes prevalence. Consequently, the demographic makeup and cultural context of each county should be considered in turn when rolling out prevention programs.

In [ ]:
sns.regplot(x = diabetes['hispanic or latino and race_total population_hispanic or latino (of any race)'],
           y = diabetes['Diabetes_Number'])

## 3. Prepare to Fit Models

### 3.1 Finalize Data Set

We've already cleaned up the data, but we can make a few more adjustments before partitioning the data and training models. Let's recode 'State' to be a categorical variable using `pd.get_dummies` and drop 'County' using `.drop()` because 'CountyFIPS' is already a unique identifier for the county. 

In [ ]:
# create dummy features out of 'State' , which might be related to diabetes rates 
diabetes_clean = pd.get_dummies(diabetes, 
                               columns = ['Region'],  
                               drop_first = True) # only create 49 dummies by dropping first in category

In [ ]:
assert diabetes_clean['race alone or in combination with one or more other races_total population'].all() == 1

In [ ]:
# drop 'County' variable
diabetes_clean = diabetes_clean.drop(labels = ['County'],
                               axis = 1) # which axis tells python we want to drop columns rather than index rows?

# look at first 10 rows of new data frame 
diabetes_clean.head(10) 

### 3.2/3.3 Partition Data and Feature Selection

Now, we will partition our data to prepare it for the training process. We will use 60% train—20% validation—20% test in this case. More data in the training set lowers bias, but then increases variance in the validation/test sets. Balancing between bias and variance with choice of these set sizes is important as we want to ensure that there is enough data to train on to get good predictions, but also want to make sure our hold-out sets are representative enough. This case is difficult since we only have a few thousand observations. It would help if we had more years worth of data.

The training set is the portion of the data we use to fit our model. The alogirthm will solve the parameters of the objective function using the observations in the training data. In the case of OLS, this is the vector $\beta$ that minimizes the residual sum of squares. Formally, we solve the following criterion function using the training data:
$$
\min_{\beta} \sum_{i=1}^n (y_i - \beta x_{i})^2
$$

The validation set is used to test how well the fitted model works with different tuning parameters. Testing the model in an interim stage helps us to assess how to set tuning parameters such as the rate of regularization in the case of Ride or LASSO in order to ensure that the fitted model generalizes well to out of sample data.

The test set is the final portion of the data used to formally assess how well our fitted algorithm performs on non-training data. This is intended to capture how well the model will work when we take it to new data

In [ ]:
# Set y 
y = diabetes_clean['Diabetes_Number']

# X (everything except diabetes, our target)
X = diabetes_clean.drop(columns = ['Diabetes_Number', 'CountyFIPS', 'State'])
features_df = diabetes_clean.drop(columns = ['Diabetes_Number', 'State', 'CountyFIPS', 'race alone or in combination with one or more other races_total population'])

In [ ]:
X

In [ ]:
X.shape

In [ ]:
selector = feature_selection.VarianceThreshold(0)
X = selector.fit_transform(X)

In [ ]:
X.shape

In [ ]:
# split the data
# train_test_split returns 4 values: X_train, X_test, y_train, y_test, so how do we create a 60-20-20 train-validate-test split? 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)

In [ ]:
#X = preprocessing.scale(X)

In [ ]:
np.mean(X[:,20])

In [ ]:
#Now split the training sample into a validation and a true training set
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = .25, random_state = 456)

We will want to standardize our data to be mean centered and have unit variance. Forcing all features to be on the same scale helps to ensure that no one feature dominates the objective function. This ensures that the algorithm can learn from all of the features 

In [ ]:
scaler = StandardScaler().fit(X_train) 

In [ ]:
X_train = scaler.transform(X_train)
X_validate = scaler.transform(X_validate)
X_test = scaler.transform(X_test)

In [ ]:
np.mean(X_train[:,5])

In [ ]:
np.mean(X_validate[:,25])

## 4. Train Models

### 4.1 Describe the selected models

### OLS
OLS fits a linear model to the data that minimizes the sum of squared errors, or the sum of squared differences between the model prediction and the actual observations. OLS assumes that the true relationship between the regressors and the independent variable is linear. OLS may poorly fit the data if the true relationship is not linear. One advantage is that the coefficients from OLS are easily interpretable. Under OLS, terms have an additive correlation with the outcome of interest and a one unit change in a regressor has the same effect on the outcome no matter the level of the regressor. 


### RIDGE
Similar to OLS, ridge regression fits a linear model (technically it does not need to be linear as you could have ridge for logit or other non-linear models). The objective function of the ridge regression is to minimize the sum of squared residuals plus a term that depends on the absolute size of each coeffcient subject to a tuning parameter. In essence, this prevents any coefficient $\beta_j$ from being too large if the relevant predictor does not explain a sufficient amount of the variation in the outcome variable. Overall, this has the effect of minimizing the coefficients on unimportant predictors which reduces overfitting. One con to ridge regression is that coefficients are less interpretable. Also, the ridge regression will include all the covariates in the final estimate instead of just including the important covariates as opposed to LASSO. Additionally, the ridge regression will introduce bias into the coefficient estimates. Some predictors may not explain a significant portion of the variation in the outcome, but nonethless may be important correlates between a specific predictor of interest and the outcome simultaneously. This will bias the causal interpretation of the coefficient on that predictor of interest.

### LASSO
The LASSO works similar to the ridge regression in that it minimizes the sum of squared residuals subject to a penalty on the size of the coefficients. But where the ridge regression penalizes the squared size of the coefficients, the LASSO penalizes the absolute size. Because the absolute value has a kink at 0, many potential predictors do not explain sufficient variation in the outcome to justify their inclusion in the model. This often results in LASSO only selecting a subset of the potential predictors. Similar to ridge, the coefficients on the LASSO are less interpretable than OLS and the use of regularization introduces extra bias as the cost of lowering the out-of-sample variance in predictions.

### 4.2 Train the models

In [ ]:
# Train OLS 
ols_reg = linear_model.LinearRegression()
ols_reg.fit(X_train, y_train)
diabetes_ols_pred = ols_reg.predict(X_validate)

ols_train_pred = ols_reg.predict(X_train)
ssr = np.sum((y_train - ols_train_pred)**2)
sst = np.sum((y_train - np.mean(y_train))**2)
manual_ols_r2 = 1 - (ssr/sst)
canned_ols_r2 = r2_score(y_train, ols_train_pred)
assert canned_ols_r2 == manual_ols_r2
manual_ols_r2

In [ ]:
#Train Lasso
lasso_reg = Lasso(alpha=.001)
lasso_reg.fit(X_train, y_train)
lasso_train_pred = lasso_reg.predict(X_train)
lasso_train_r2 = r2_score(y_train, lasso_train_pred)
lasso_train_r2

In [ ]:
diabetes_lasso_pred = lasso_reg.predict(X_validate)

In [ ]:
# Find the non-zero coefficients and see what variables are being selected
features_df.iloc[:,lasso_reg.coef_!=0].columns

In [ ]:
#Train Ridge 
ridge_reg = Ridge(alpha=.001)
ridge_reg.fit(X_train, y_train)
ridge_train_pred = ridge_reg.predict(X_train)
ridge_train_r2 = r2_score(y_train, ridge_train_pred)
ridge_train_r2

In [ ]:
diabetes_ridge_pred = ridge_reg.predict(X_validate)

## 5. Validate and Refine Models

### 5.1 Test the models on the validation set 
Manually calculate the test MSE then use the canned sklearn function to check

In [ ]:
#OLS
manual_ols_validate_mse = np.mean((y_validate - diabetes_ols_pred)**2)
canned_ols_validate_mse = mean_squared_error(y_validate, diabetes_ols_pred)
assert manual_ols_validate_mse == canned_ols_validate_mse
canned_ols_validate_mse

In [ ]:
#LASSO 
manual_lasso_validate_mse = np.mean((y_validate - diabetes_lasso_pred)**2)
canned_lasso_validate_mse = mean_squared_error(y_validate, diabetes_lasso_pred)
assert manual_lasso_validate_mse == canned_lasso_validate_mse
canned_lasso_validate_mse

In [ ]:
#Ridge
manual_ridge_validate_mse = np.mean((y_validate - diabetes_ridge_pred)**2)
canned_ridge_validate_mse = mean_squared_error(y_validate, diabetes_ridge_pred)
assert manual_ridge_validate_mse == canned_ridge_validate_mse
canned_ridge_validate_mse

The lasso had the lowest mean squared error in the validation set. OLS, Ridge, and LASSO all performed comparably which is unsurprising given that we didn't add interaction terms or nonlinear terms. Adding these features may have caused ols to overfit the training data and perform poorly out of sample. Adding polynomials and interaciton terms would have made this a problem more suited to the regularization approaches of LASSO and Ridge that help to prevent overfitting by constraining the flexibility of the model. Since these features were not included and lambda was set very low for both the Ridge and LASSO, we should expect these approaches to perform similarly to OLS.

That LASSO performed slightly better than OLS out of sample is an indication that OLS slightly overfit the training sample relative to LASSO.

### 5.2 Test Set

I'm a little confused by this part. Why would we select out the unimportant features and retrain the model? This is already what LASSO is doing. What does this add?

In [ ]:
# Remove unimportant features
X_train_selected = X_train[:,lasso_reg.coef_!=0]

In [ ]:
#Re-train Lasso
lasso_reg_selected = Lasso(alpha=.001)
lasso_reg_selected.fit(X_train_selected, y_train)
lasso_selected_pred = lasso_reg.predict(X_test)

In [ ]:
#Calculate test MSE
lasso_test_mse = mean_squared_error(y_test, lasso_selected_pred)
lasso_test_mse

In [ ]:
#Evaluate test R^2
lasso_test_r2 = r2_score(y_test, lasso_selected_pred)
lasso_test_r2

Our LASSO specification can explain a little under 48% of the variation in diabetes prevalence by county in the test data set. This is a significant amount of the overall variation given the limited number of possible features. One advantage of using both the validation and test sets in a public policy application is that the cost of implementing a policy based on an innacruate algorithm may be quite high. If the policymaker is risk-averse, we want to be confident that our predictions will generalize well to other settings. Using both the test and validation sets helps to ensure that our out of sample error rate is adequate enough to justify using the algorithm for policy decisions.

### 5.3 Implement a Cross-Validation Approach

I will use cross validation to both select the value of the LASSO tuning parameter (lambda) a validation error rate. Implementing a CV approach to find the best lambda is a bad idea if we are concerned about specifying the correct model or saying something causal about diabetes prevalance. This is because a CV-selected lambda will optimize the prediction power of the model, but may shrink coefficients on important variables to zero. Here, we are only concerned with predicting the rate of diabetes in a county. Using a cross-validation approach to select the tuning parameter is appropriate.

Because we are using cross validation, we can take advantage of the full size of the training set. Rather than split the data into dedicated train/validate/test partiitions we can just perform a train/test split. The cross validation approach will take advantage of the full set of training data to both train and validate the model

In [ ]:
X_cv_train, X_cv_test, y_cv_train, y_cv_test = train_test_split(X, y, test_size=.2, random_state=123)

In [ ]:
#Need to re-standardize
cv_scaler = StandardScaler().fit(X_cv_train) 
X_cv_train = cv_scaler.transform(X_cv_train)
X_cv_test = cv_scaler.transform(X_cv_test)

In [ ]:
X_cv_test.shape

In [ ]:
y_cv_test

### Choosing number of folds for cross validation
The choice of folds when performing k-fold cross validation is a trade-off between bias and variance. The more folds there are, the fewer observations there will be in each fold. When the data is trained on K-1 folds with large k, the k-1 folds will contain most of the training data. Containing more data means that the bias will be lower as the algorithm can more accurately fit the data. However, this means that overfitting may be a problem as large k more closely approximates a leave-one-out cross validation approach where prediction on the single observation that is left out may suffer from high variance as any single observation may not be typical of the underlying distribtion of data.

This is less of a problem with larger datasets where each fold will contain sufficiently many observations to be approximately representative of the uderlying distribution. 


In [ ]:
lasso_cv_specification = LassoCV(cv=5, random_state=0, max_iter=10000)
lasso_cv_specification.fit(X_cv_train, y_cv_train)

In [ ]:
best_alpha = lasso_cv_specification.alpha_
best_alpha

In [ ]:
cv_lasso = Lasso(alpha = best_alpha)
cv_lasso.fit(X_cv_train, y_cv_train)
cv_lasso_train_pred = lasso_reg.predict(X_cv_train)
cv_lasso_test_pred = lasso_reg.predict(X_cv_test)

In [ ]:
# Find the non-zero coefficients and see what variables are being selected
features_df.iloc[:,cv_lasso.coef_!=0].columns

In [ ]:
# Get the training R^2 (in a slightly different way for practice)
train_mse = round(mean_squared_error(y_cv_train, cv_lasso_train_pred), 6)
print("The training set mean squared error in the cross validation approach is:", train_mse)

In [ ]:
train_r2 = round(cv_lasso.score(X_cv_train, y_cv_train)*100, 2)
print("The training set R-Squared in the cross validation approach is:", train_r2)

In [ ]:
#Evaluate performance in test set
test_mse = round(mean_squared_error(y_cv_test, cv_lasso_test_pred), 6)
print("The test set mean squared error in the cross validation approach is:", test_mse)

test_r2 = round(cv_lasso.score(X_cv_test, y_cv_test)*100, 2)
print("The test set R-Squared in the cross validation approach is:", test_r2)

In [ ]:
coef_dict = {
    "Feature": list(features_df.columns),
    "Coefficient": cv_lasso.coef_
}

In [ ]:
# Create a DataFrame and sort by coefficient magnitude in descending order
coefficients_df = pd.DataFrame(coef_dict)
coefficients_df = coefficients_df.reindex(coefficients_df["Coefficient"].abs().sort_values(ascending=False).index)

In [ ]:
coefficients_df

## 6. Discussion Questions

### 6.1 What is the bias-variance tradeoff? Why is it relevant to machine learning poblems like this one?
You can decrease bias, or the degree to which the fitted values of your model differ from the actual values, by incorporating more information, or predictors, into your model. Doing this in a sample, there is some degree to which you are capturing the way the real world works, and some degree to which you are explaining random variation in your particular sample. If you include lots of predictors that explain random variation, then your predcition model will not be consitent when you use it on other samples since those new samples will have random variation that looks very different from your original data. This is is the bias-variance trade-off. This is relevant to machine learning because we care about creating prediction models that capture the way the world truly works, and that give us consistent and accurate predictions across models.

### 6.2 Define overfitting, and why it matters for machine learning. How can we address it?
Overfitting is tailoring your estimate of a model to fit the idiosyncracies of a particular dataset that are a product of random error or potentially measurment error rather than the underlying data generating process. In the context of diabetes, we might think of diabetes as caused primarily by diet, exercise, and genetic factors. However, we may, by random chance, get a sample of data on diabetes prevalence that happens to contain lots of individuals who are accounttants and have diabetes. This does not mean that diabetes is caused by becoming an accountant. This is just a feature of the particular sample we happened to draw. Overfitting in this particular example means that we fit a model that uses being an accountant as a strong predictor of diabetes. 

In machine learning, we are primarily concenred with prediction. In our toy example, our model is unlikely to predict diabetes well when we apply it to other datasets because being an accountant has no direct link with diabetes. In practice, the random noise we get in any given dataset is likely to be more subtle than the accountant example. Nevertheless, our algorithm may chase down this random noise rather than focusing on information that truly predicts our outcome.

One simple way to address overfitting is to average models over many many samples. If we take multiple samples, the idisyncracies from any one sample will eventually cancel each other out. Some samples may have lots of accountants with diabetes leading to a model that uses being an accountant as an improtant predictor of diabetes. Other samples may have very few accountants and will conclude this is not an improtant predictor. Other samples may have lots of accountants who do not have diabetes and conclude being an accountant is a protective factor. By averaging these the predictions across these samples together, the noise (being an accountant) will not being given much weight. 

Another way to prevent overfitting is to only place weight on predictors that expalin a significant amount of variation in the outcome, this is regularization. Being an accountant may explain some variation in diabetes in a given sample, but it is unlikely to explain that much. Regularization will help to produce a model where predictors that explain small pockets of noise are given little to no improtance for prediction.



### 6.3 Discussion of analysis
According to the algorithm, predictors like the obesity and physcial activity number, as well as whether the county is in the South or Southeast are indicative of higher diabetes rates. It would be natural to pilot the programs in these areas. However, we know the actual diabetes rate and we are interested in preventing diabetes. It would likely be best to prioritize piloting the program in areas where obesity and physical inactivity are increasing, but where diabetes is not necessarily high as these factors may be leading predictors of diabetes. However, the algorithm does not tell us this and certainly does not tell us the causal relationship between diabetes and these factors. 

Race, gender, and age are important predictors of diabetes rates according to the algorithm, the ommision or inclusion of these predictors would likely change how accurate the algorithm is at predicting diabetes rates. While this algorithm is a good start for predciting diabetes rates, it does not predict future diabetes rates or factors that cause diabetes that might be changeable through the prevention program. Consequently, it would be better to run this algorithm on panel data and to know more about the design of the prevention problem. That being said, if we truly need to a contemporaneous predictor of diabetes rates, this algorithm may be our best availble option.